In [ ]:
# Imports
import os
from glob import glob
import random

import numpy as np
import pandas as pd
import cv2 as cv
from PIL import Image
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

#serene's imports
from skimage import data, io, filters
import math

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF

# from __future__ import division
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import sys
import numpy as np
from numpy import linalg
from random import random

from sklearn import model_selection
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import svm

In [ ]:
# vim: set ts=4 sts=4 sw=4 expandtab smartindent:
#
# This file was originally part of the octave-forge project
# Ported to python by Luis Pedro Coelho <luis@luispedro.org> (February 2008)
# Copyright (C) 2006       Soren Hauberg
# Copyright (C) 2008-2010  Luis Pedro Coelho (Python port)
# 
# This program is free software; you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation; either version 2, or (at your option)
# any later version.
# 
# This program is distributed in the hope that it will be useful, but
# WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
# General Public License for more details. 
# 
# You should have received a copy of the GNU General Public License
# along with this file.  If not, see <http://www.gnu.org/licenses/>.



__all__ = ['bwperim']

def bwperim(bw, n=4):
    """
    perim = bwperim(bw, n=4)
    Find the perimeter of objects in binary images.
    A pixel is part of an object perimeter if its value is one and there
    is at least one zero-valued pixel in its neighborhood.
    By default the neighborhood of a pixel is 4 nearest pixels, but
    if `n` is set to 8 the 8 nearest pixels will be considered.
    Parameters
    ----------
      bw : A black-and-white image
      n : Connectivity. Must be 4 or 8 (default: 8)
    Returns
    -------
      perim : A boolean image
    """

    if n not in (4,8):
        raise ValueError('mahotas.bwperim: n must be 4 or 8')
    rows,cols = bw.shape

    # Translate image by one pixel in all directions
    north = np.zeros((rows,cols))
    south = np.zeros((rows,cols))
    west = np.zeros((rows,cols))
    east = np.zeros((rows,cols))

    north[:-1,:] = bw[1:,:]
    south[1:,:]  = bw[:-1,:]
    west[:,:-1]  = bw[:,1:]
    east[:,1:]   = bw[:,:-1]
    idx = (north == bw) & \
          (south == bw) & \
          (west  == bw) & \
          (east  == bw)
    if n == 8:
        north_east = np.zeros((rows, cols))
        north_west = np.zeros((rows, cols))
        south_east = np.zeros((rows, cols))
        south_west = np.zeros((rows, cols))
        north_east[:-1, 1:]   = bw[1:, :-1]
        north_west[:-1, :-1] = bw[1:, 1:]
        south_east[1:, 1:]     = bw[:-1, :-1]
        south_west[1:, :-1]   = bw[:-1, 1:]
        idx &= (north_east == bw) & \
               (south_east == bw) & \
               (south_west == bw) & \
               (north_west == bw)
    return ~idx * bw

In [ ]:
# SERENE'S CODE

def find_centroid(mask1):
    M = cv.moments(mask1) #all three parts of the mask are identical

    # calculate x,y coordinate of center

    X = int(M["m10"] / M["m00"])
    Y = int(M["m01"] / M["m00"]) 

    return X,Y

In [ ]:
def std_one_bin(angle_vals, og_inds1, og_inds2, starting_angle, ending_angle, mean1, X, Y):
    
    
    inds1 = og_inds1[ angle_vals < ending_angle ]
    inds2 = og_inds2[ angle_vals < ending_angle ]
    angle_vals2 = angle_vals[angle_vals < ending_angle ]
    inds1 = inds1[ angle_vals2 >= starting_angle ]
    inds2 = inds2[ angle_vals2 >= starting_angle ]
    
#     print(inds1)
#     print(inds2)
#     print(angle_vals2[angle_vals2 > starting_angle])
    
    radial_dists = np.sqrt(np.square((inds1 - Y)) + np.square(inds2 - X))/mean1
    if (len((radial_dists))>0):
        return np.std( radial_dists )
    else:
        return 0

In [ ]:
def ten_equiangular_bins(edges_all, x_center, y_center, mean_rad_dist):
    final_bin_stds = np.empty((10,1))
    
    edges_top = edges_all[0:y_center-1 ,:]
    [top_inds1, top_inds2] = np.where(edges_top >0.1)
    
    starting_vec = [1, 0] 
    starting_vec = starting_vec / np.linalg.norm(starting_vec)
    
    cos_vals = np.zeros(len(top_inds1))
    for t in range(len(top_inds1)):
        curr_vec = [top_inds2[t] - x_center, top_inds1[t] - y_center ] 
        curr_vec = curr_vec / np.linalg.norm(curr_vec)
        cos_vals[t] = np.dot(starting_vec, curr_vec)
    
    pre1 = np.argsort(cos_vals)
    top_inds1 = top_inds1[pre1]
    top_inds2 = top_inds2[pre1]       
#     print(cos_vals[pre1])
    
    
    final_bin_stds[0] = std_one_bin( cos_vals, top_inds1, top_inds2, 0.809017,1 , mean_rad_dist, x_center, y_center )
    final_bin_stds[1] = std_one_bin( cos_vals, top_inds1, top_inds2, 0.309017,0.809017 , mean_rad_dist, x_center, y_center )
    final_bin_stds[2] = std_one_bin( cos_vals, top_inds1, top_inds2, -0.309017,0.309017 , mean_rad_dist, x_center, y_center )
    final_bin_stds[3] = std_one_bin( cos_vals, top_inds1, top_inds2, -0.809017, -0.309017, mean_rad_dist, x_center, y_center )
    final_bin_stds[4] = std_one_bin( cos_vals, top_inds1, top_inds2, -1, -0.809017, mean_rad_dist, x_center, y_center )



    edges_bottom = edges_all[y_center:np.size(edges_all,0)-1 ,:]
    [bottom_inds1, bottom_inds2] = np.where(edges_bottom >0.1)
    bottom_inds1 = bottom_inds1 + y_center
    cos_vals = np.zeros(len(bottom_inds1))
    for t in range(len(bottom_inds1)):
        curr_vec = [bottom_inds2[t] - x_center, bottom_inds1[t] - y_center ]  # don't need to offset y_center bc already done by cropping
        curr_vec = curr_vec / np.linalg.norm(curr_vec)
        cos_vals[t] = np.dot( curr_vec, starting_vec )
        
    
    pre2 = np.argsort(cos_vals)
    order2 = np.flip(pre2)
    bottom_inds1 = bottom_inds1[order2]
    bottom_inds2 = bottom_inds2[order2]
#     print(cos_vals[order2])

    final_bin_stds[5] = std_one_bin( cos_vals, bottom_inds1, bottom_inds2, 0.809017,1 , mean_rad_dist, x_center, y_center )
    final_bin_stds[6] = std_one_bin( cos_vals, bottom_inds1, bottom_inds2, 0.309017,0.809017 , mean_rad_dist, x_center, y_center )
    final_bin_stds[7] = std_one_bin( cos_vals, bottom_inds1, bottom_inds2, -0.309017,0.309017 , mean_rad_dist, x_center, y_center )
    final_bin_stds[8] = std_one_bin( cos_vals, bottom_inds1, bottom_inds2, -0.809017, -0.309017, mean_rad_dist, x_center, y_center )
    final_bin_stds[9] = std_one_bin( cos_vals, bottom_inds1, bottom_inds2, -1, -0.809017, mean_rad_dist, x_center, y_center )


    return final_bin_stds

In [ ]:
# ASSUME THAT mask IS A 2D BINARY IMAGE OF WHERE THE TUMOR IS

# OUTPUTS: False, 0 if no tumor in layer
# True if tumor in layer

# BASED ON ONE 2017 PAPER, ONLY NEED TO CALCULATE FOR THE "images with the largest tumor cross section for each patient." 

def angular_std(mask):
    if np.sum(np.sum(mask))==0:
        return False
    else:
        X,Y = find_centroid(mask)
#         print(mask[Y,X]) # how to access centroid pt
#         print(X,Y)
        
#         edges = filters.scharr(curr_im[:,:,0])
        edges = bwperim(mask)
        plt.figure()
        plt.imshow(edges) 
        plt.scatter(X,Y, color='r')

#         print(edges[119, 184])
        [inds1, inds2] = np.where(edges >0.1)

        radial_dists = np.sqrt(np.square((inds1 - Y)) + np.square(inds2 - X)) # DON'T WORRY, checked that inds1 ~ Y and inds2 ~ X
#         print(radial_dists)
        mean_dists = np.mean(radial_dists)        
        radial_dists = radial_dists/mean_dists 
        
        Rs_10 = ten_equiangular_bins(edges, X, Y, mean_dists)
#         print(Rs_10)
        final_ans = np.mean(Rs_10)
        if np.isnan(final_ans):
            final_ans =0
        return final_ans

        


In [ ]:
def sort_edges(edges_all, x_center, y_center):
    edges_top = edges_all[0:y_center-1 ,:]
    [top_inds1, top_inds2] = np.where(edges_top >0.1)
    
    starting_vec = [1, 0] 
    starting_vec = starting_vec / np.linalg.norm(starting_vec)
    
    cos_vals = np.zeros(len(top_inds1))
    for t in range(len(top_inds1)):
        curr_vec = [top_inds2[t] - x_center, top_inds1[t] - y_center ] 
        curr_vec = curr_vec / np.linalg.norm(curr_vec)
        cos_vals[t] = np.dot(starting_vec, curr_vec)
    
    pre1 = np.argsort(cos_vals)
    top_inds1 = top_inds1[pre1]
    top_inds2 = top_inds2[pre1]       
#     print(cos_vals[pre1])

    edges_bottom = edges_all[y_center:np.size(edges_all,0)-1 ,:]
    [bottom_inds1, bottom_inds2] = np.where(edges_bottom >0.1)
    bottom_inds1 = bottom_inds1 + y_center
    cos_vals = np.zeros(len(bottom_inds1))
    for t in range(len(bottom_inds1)):
        curr_vec = [bottom_inds2[t] - x_center, bottom_inds1[t] - y_center ]  # don't need to offset y_center bc already done by cropping
        curr_vec = curr_vec / np.linalg.norm(curr_vec)
        cos_vals[t] = np.dot( curr_vec, starting_vec )
        
    
    pre2 = np.argsort(cos_vals)
    order2 = np.flip(pre2)
    bottom_inds1 = bottom_inds1[order2]
    bottom_inds2 = bottom_inds2[order2]
#     print(cos_vals[order2])

    inds1 = np.concatenate((top_inds1, bottom_inds1))
    inds2 = np.concatenate((top_inds2, bottom_inds2))
    return inds1, inds2

In [ ]:
def margin_fluctuation(mask):
    if np.sum(np.sum(mask))==0:
        return False
    else:
        X,Y = find_centroid(mask)
        
#         print(X,Y)

#         [inds1, inds2] = np.where(edges >0.1)
#         print(np.size(inds1))
        
        # want the order of indices to go radially rather than zigzagging over rows
        
        edges = bwperim(mask , 4)
        y_inds, x_inds = sort_edges(edges, X, Y)
        
        radial_dists = np.sqrt(np.square((y_inds - Y)) + np.square(x_inds - X))
        mean_dists = np.mean(radial_dists)        
        radial_dists = radial_dists/mean_dists # normalize based on the avg radial distance to the centroid
        
        
        p = int(0.1*len(y_inds)) # 10% of the length of the perimeter

        padded_rs = np.concatenate(((radial_dists[len(radial_dists)-p:len(radial_dists)]),radial_dists))
        padded_rs = np.concatenate((padded_rs, radial_dists[0:p]))
        avged_radial_dists = np.convolve(padded_rs, np.ones(p)/p, mode='same') 
        avged_radial_dists = avged_radial_dists[p:len(padded_rs)-p] #Take away the padding that was originally used

        
        diff1 = avged_radial_dists - radial_dists #[p:len(padded_rs)-p]
        
        plt.figure()
        plt.plot( range(len(radial_dists)), radial_dists,range(len(avged_radial_dists)), avged_radial_dists )
        
        stdev_all = np.std(diff1)
        
        
        return stdev_all

In [ ]:
# CREDIT TO: https://github.com/minillinim/ellipsoid/blob/master/ellipsoid.py 

class EllipsoidTool:
    """Some stuff for playing with ellipsoids"""
    def __init__(self): pass
    
    def getMinVolEllipse(self, P=None, tolerance=0.01):
        """ Find the minimum volume ellipsoid which holds all the points
        
        Based on work by Nima Moshtagh
        http://www.mathworks.com/matlabcentral/fileexchange/9542
        and also by looking at:
        http://cctbx.sourceforge.net/current/python/scitbx.math.minimum_covering_ellipsoid.html
        Which is based on the first reference anyway!
        
        Here, P is a numpy array of N dimensional points like this:
        P = [[x,y,z,...], <-- one point per line
             [x,y,z,...],
             [x,y,z,...]]
        
        Returns:
        (center, radii, rotation)
        
        """
        (N, d) = np.shape(P)
        d = float(d)
    
        # Q will be our working array
        Q = np.vstack([np.copy(P.T), np.ones(N)]) 
        QT = Q.T
        
        # initializations
        err = 1.0 + tolerance
        u = (1.0 / N) * np.ones(N)

        # Khachiyan Algorithm
        while err > tolerance:
            V = np.dot(Q, np.dot(np.diag(u), QT))
            M = np.diag(np.dot(QT , np.dot(linalg.inv(V), Q)))    # M the diagonal vector of an NxN matrix
            j = np.argmax(M)
            maximum = M[j]
            step_size = (maximum - d - 1.0) / ((d + 1.0) * (maximum - 1.0))
            new_u = (1.0 - step_size) * u
            new_u[j] += step_size
            err = np.linalg.norm(new_u - u)
            u = new_u

        # center of the ellipse 
        center = np.dot(P.T, u)
    
        # the A matrix for the ellipse
        A = linalg.inv(
                       np.dot(P.T, np.dot(np.diag(u), P)) - 
                       np.array([[a * b for b in center] for a in center])
                       ) / d
                       
        # Get the values we'd like to return
        U, s, rotation = linalg.svd(A)
        radii = 1.0/np.sqrt(s)
        
        return (center, radii, rotation)

    def getEllipsoidVolume(self, radii):
        """Calculate the volume of the blob"""
        return 4./3.*np.pi*radii[0]*radii[1]*radii[2]

    def plotEllipsoid(self, center, radii, rotation, ax=None, plotAxes=False, cageColor='b', cageAlpha=0.2):
        """Plot an ellipsoid"""
        make_ax = ax == None
        if make_ax:
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            
        u = np.linspace(0.0, 2.0 * np.pi, 100)
        v = np.linspace(0.0, np.pi, 100)
        
        # cartesian coordinates that correspond to the spherical angles:
        x = radii[0] * np.outer(np.cos(u), np.sin(v))
        y = radii[1] * np.outer(np.sin(u), np.sin(v))
        z = radii[2] * np.outer(np.ones_like(u), np.cos(v))
        # rotate accordingly
        for i in range(len(x)):
            for j in range(len(x)):
                [x[i,j],y[i,j],z[i,j]] = np.dot([x[i,j],y[i,j],z[i,j]], rotation) + center
    
        if plotAxes:
            # make some purdy axes
            axes = np.array([[radii[0],0.0,0.0],
                             [0.0,radii[1],0.0],
                             [0.0,0.0,radii[2]]])
            # rotate accordingly
            for i in range(len(axes)):
                axes[i] = np.dot(axes[i], rotation)
    
    
            # plot axes
            for p in axes:
                X3 = np.linspace(-p[0], p[0], 100) + center[0]
                Y3 = np.linspace(-p[1], p[1], 100) + center[1]
                Z3 = np.linspace(-p[2], p[2], 100) + center[2]
                ax.plot(X3, Y3, Z3, color=cageColor)
    
        # plot ellipsoid
        ax.plot_wireframe(x, y, z,  rstride=4, cstride=4, color=cageColor, alpha=cageAlpha)
        
        if make_ax:
            plt.show()
            plt.close(fig)
            del fig
        



In [ ]:
# Bounding Ellipsoid Volume Ratio Calculation

def bevr(sorted_mask_list):
    z_factor = 2; # the ratio of slice thickness to x-y pixel size
    
    # Build set of points 
    all_points = []
    tumor_vol = 0;
    for f in range(len(sorted_mask_list)): 
        curr_im = cv.imread(sorted_mask_list[f], cv.IMREAD_GRAYSCALE)
        mask_pts = np.nonzero(curr_im)
        
        edges = bwperim(curr_im, 4)
        edge_pts = np.nonzero(edges)
        if np.size(edge_pts) > 0:
            xs = edge_pts[0]
            ys = edge_pts[1]
            tumor_vol = tumor_vol + z_factor*len(mask_pts[0]) # Surface Area = number of points, Volume = z * surface area
            for m in range(len(xs)):
                pt_curr = [xs[m], ys[m], z_factor*f] 
                if np.size(all_points) > 0:
                    all_points = np.append(all_points, pt_curr)
                else:
                    all_points = pt_curr

    l = int(len(all_points)/3)
    all_points = np.reshape(all_points,(l, 3))
    ET = EllipsoidTool()
    (center, radii, rotation) = ET.getMinVolEllipse(all_points, .01)
    vol_ellipsoid =  ET.getEllipsoidVolume(radii)
#     print(vol_ellipsoid)
    
    # Plotting code
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(all_points[:,0], all_points[:,1], all_points[:,2], color='g', marker='*', s=100)
    ET.plotEllipsoid(center, radii, rotation, ax=ax, plotAxes=True)
    plt.show()
    plt.close(fig)
    
    final_ratio = tumor_vol/vol_ellipsoid
    return final_ratio

In [ ]:
df = pd.read_csv("/kaggle/input/lgg-mri-segmentation/lgg-mri-segmentation/kaggle_3m/data.csv")
labels = df.COCCluster.to_numpy()
labels -= 1
print(np.unique(labels))

patients = []
data_dir = '/kaggle/input/lgg-mri-segmentation/kaggle_3m/'
for dirname, _, filenames in os.walk(data_dir):
    if dirname == data_dir:
        continue        
    patients.append(dirname)


In [ ]:
features = np.empty((1, 3))
idx = 0;
patient = patients[2]
files = os.listdir(patient)

sorted_names = [None]*int(len(files)/2)
largest_tumor_section = np.zeros((1, 256, 256))

for i in files:
    if "mask" in i:
        fn = patient + "/" + i              
        index = int(i.split("_")[-2]) - 1
        sorted_names[index] = fn 

        m = cv.imread(fn, cv.IMREAD_GRAYSCALE)    
        curr_nonzs = np.size(np.nonzero(m))
        old_nonzs = np.size(np.nonzero(largest_tumor_section))

        if curr_nonzs > old_nonzs:
            largest_tumor_section = m      

 # Only need to calculate 2D features from the largest tumor cross-section for each patient
features[idx,0] = angular_std(largest_tumor_section)
features[idx,1] = margin_fluctuation(largest_tumor_section)
features[idx,2] = (bevr(sorted_names))   
print(features[idx,:])

In [ ]:


features = np.empty((len(patients), 3))
idx = 0;
for patient in patients:
    files = os.listdir(patient)

    sorted_names = [None]*int(len(files)/2)
    largest_tumor_section = np.zeros((1, 256, 256))
    
    for i in files:
        if "mask" in i:
            fn = patient + "/" + i              
            index = int(i.split("_")[-2]) - 1
            sorted_names[index] = fn 
            
            m = cv.imread(fn, cv.IMREAD_GRAYSCALE)    
            curr_nonzs = np.size(np.nonzero(m))
            old_nonzs = np.size(np.nonzero(largest_tumor_section))
            
            if curr_nonzs > old_nonzs:
                largest_tumor_section = m      
            
     # Only need to calculate 2D features from the largest tumor cross-section for each patient
    features[idx,0] = angular_std(largest_tumor_section)
#     features[idx,1] = margin_fluctuation(largest_tumor_section)
#     features[idx,2] = (bevr(sorted_names))   
#     print(features[idx,:])
    
    idx = idx+1;

In [ ]:
np.save('features_all_og.npy', features)
np.save('labels.npy', labels)

In [ ]:
feats = np.load('../input/og-lgg-outputs/features_all_og.npy')
labs = np.load('../input/og-lgg-outputs/labels.npy')

In [ ]:
## Custom linear regression 
# X: input features
# y: data labels
# spl: the chosen instance of ShuffleSplit or other splitter used

def train_lin_regr_custom(X,y, spl):    
    y = y.ravel() # slightly modifies the shape of y    
    curr_min_test_acc = 1e6  # will be set to the current lowest test accuracy    
    best_params = [] # the ideal feature amount corresponding to lowest test accuracy will be saved here
    
    
    tr_accs = []
    test_accs = []
    for train_index, test_index in spl.split(X,y):    
        # Split up data into test and train data
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index] 

        # Normalization can help Linear Regression accuracy  
        scl = MinMaxScaler() # subtract X_min then divide by (X_max - X_min)
        X_train = scl.fit_transform(X_train) # set the mean + std for each feature and apply the scaler 
        X_test = scl.transform(X_test) #apply the scaler

        # Initialize and fit model
        model = LinearRegression(fit_intercept = True)
        model = model.fit(X_train, y_train)

        # Find model performance on training data
        y_pred = model.predict(X_train)
        tr1 = sum(y_train == np.round(y_pred))/len(y_train)

        # Find model performance on testing data and append to lists
        y_pred = model.predict(X_test)
        te1 = sum(y_test == np.round(y_pred))/len(y_test)

        tr_accs.append((tr1))
        test_accs.append((te1)) 


    # Average test error across all n_splits
    avg_te = sum(test_accs)/len(test_accs)

    # Update "best" feature amount if testing RMSE is improved
    if (curr_min_test_acc > avg_te):
        curr_min_test_acc = avg_te

#     # Print model results 
    print("AVG Testing:", np.round(avg_te,3), 
                  " AVG Training: ", np.round(sum(tr_accs)/len(tr_accs),3))

    
    print("\nTrain size", X_train.shape)
    print("Test size", X_test.shape)
    # print(model.get_params())
    return test_accs

In [ ]:
## Custom SVM Classifier 
# X: input features
# y: data labels
# spl: the chosen instance of ShuffleSplit or other splitter used

def train_SVM_custom(X,y, spl, c_custom):    
    y = y.ravel() # slightly modifies the shape of y    
    curr_min_test_acc = 1e6  # will be set to the current lowest test accuracy    
    best_params = [] # the ideal feature amount corresponding to lowest test accuracy will be saved here
    
    
    tr_accs = []
    test_accs = []
    for train_index, test_index in spl.split(X,y):    
        # Split up data into test and train data
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index] 

        # Normalization can help   accuracy  
        scl = StandardScaler() 
        X_train = scl.fit_transform(X_train) # set the mean + std for each feature and apply the scaler 
        X_test = scl.transform(X_test) #apply the scaler

        # Initialize and fit model
        model = svm.SVC(C=c_custom)
        model = model.fit(X_train, y_train)

        # Find model performance on training data
        y_pred = model.predict(X_train)
        tr1 = sum(y_train == np.round(y_pred))/len(y_train)

        # Find model performance on testing data and append to lists
        y_pred = model.predict(X_test)
        te1 = sum(y_test == np.round(y_pred))/len(y_test)

        tr_accs.append((tr1))
        test_accs.append((te1)) 


    # Average test error across all n_splits
    avg_te = sum(test_accs)/len(test_accs)

    # Update "best" feature amount if testing RMSE is improved
    if (curr_min_test_acc > avg_te):
        curr_min_test_acc = avg_te

#     # Print model results 
    print("AVG Testing:", np.round(avg_te,3), 
                  " AVG Training: ", np.round(sum(tr_accs)/len(tr_accs),3))

    
    print("\nTrain size", X_train.shape)
    print("Test size", X_test.shape)
    # print(model.get_params())
    return test_accs

In [ ]:
rand_seed = 10;
spl1 = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=rand_seed)
all_kfold_accuracies = train_lin_regr_custom(feats,labs, spl1)
print(all_kfold_accuracies)
all_kfold_accuracies = train_SVM_custom(feats,labs, spl1,1000)
print(all_kfold_accuracies)